In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import numpy as np
import timber_nds.settings as settings
import timber_nds.calculation as calculation
import timber_nds.design as design

# Parameters

**Material Properties:**

* `species`: Wood species.
* `specific_gravity`: Specific gravity of the wood.
* `fibre_saturation_point`: Fiber saturation point of the wood (%).
* `moisture_content`: Moisture content of the wood (%).
* `e_modulus`: Modulus of elasticity of the wood (kgf/cm2).
* `fb`: Bending strength (kgf/cm2).
* `ft`: Tensile strength (kgf/cm2).
* `fv`: Shear strength (kgf/cm2).
* `fc`: Compression strength parallel to the grain (kgf/cm2).
* `fc_perp`: Compression strength perpendicular to the grain (kgf/cm2).

In [3]:
species = "Douglas Fir-Larch"
specific_gravity = 0.55
fibre_saturation_point = 28
moisture_content = 19 #19%
e_modulus = 115000 # kgf/cm2
fb = 280  # kgf/cm2
ft = 280  # kgf/cm2
fv = 45  # kgf/cm2
fc = 280  # kgf/cm2
fc_perp = 28  # kgf/cm2

In [4]:
wood_material = settings.WoodMaterial(
    name=species,
    specific_gravity=specific_gravity,
    fibre_saturation_point=fibre_saturation_point,
    tension_strength=ft,
    bending_strength=fb,
    shear_strength=fv,
    compression_perpendicular_strength=fc_perp,
    compression_parallel_strength=fc,
    elastic_modulus=e_modulus,
    color="Brown"
)

**Geometric Dimensions:** 

* `width`: Width of the cross-section (cm).
* `depth`: Depth of the cross-section (cm).
* `length`: Length of the element (cm).
* `k_factor`: Effective length factor.


In [5]:
width = 5. # cm
depth = 15. # cm
length = 400 # cm
k_factor = 1.0

section = settings.RectangularSection("sección 1", depth=depth, width=width)
element = settings.MemberDefinition("elemento 1", length=length, )

**All Adjustment Factors:**

* Several adjustment factors are defined for tension  `TensionAdjustmentFactors`, bending `BendingAdjustmentFactors`, shear `ShearAdjustmentFactors`, compression `CompressionAdjustmentFactors`, compression perpendicular to the grain `PerpendicularAdjustmentFactors`, and modulus of elasticity `ElasticModulusAdjustmentFactors`.

**Strength Reduction Factors:**

* `phi_tension`: Strength reduction factor for tension.
* `phi_bending`: Strength reduction factor for bending.
* `phi_shear`: Strength reduction factor for shear.
* `phi_compression`: Strength reduction factor for compression.
* `phi_perp_compression`: Strength reduction factor for compression perpendicular to the grain.

In [6]:
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

**Adjustment Factors:**

* `TensionAdjustmentFactors`: Factors to adjust the basic tensile stress.
* `BendingAdjustmentFactors`: Factors to adjust the basic bending stress.
* `ShearAdjustmentFactors`: Factors to adjust the basic shear stress.
* `CompressionAdjustmentFactors`: Factors to adjust the basic compression stress parallel to the grain.
* `PerpendicularAdjustmentFactors`: Factors to adjust the basic compression stress perpendicular to the grain.
* `ElasticModulusAdjustmentFactors`: Factors to adjust the modulus of elasticity.

In [7]:
ft_factors = settings.TensionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [8]:
fb_factors_yy = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0, 
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [9]:
fb_factors_zz = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [10]:
fv_factors = settings.ShearAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.97,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [11]:
fc_factors_yy = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [12]:
fc_factors_zz = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [13]:
fc_perp_factors = settings.PerpendicularAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.67,
    due_temperature=1.0,
    due_bearing_area=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

In [14]:
e_factors = settings.ElasticModulusAdjustmentFactors(
    due_moisture=0.90,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0
)

# Example


This example will demonstrate the calculation process for the weight, geometric properties, and capacity of a rectangular timber beam.


## density and weight calculation

This section demonstrates how to calculate the weight of a timber element using the `WeightCalculator` class.
First, an instance of `WoodMaterial`, `RectangularSection`, and `MemberDefinition` is created with the parameters defined previously.
Then, an instance of `WeightCalculator` is created, and the weight of the element is calculated for the specified moisture content.

In [44]:
weight_calculator = calculation.WeightCalculator(wood_material, section, element)
material_density = weight_calculator.calculate_density_at_moisture_content(moisture_content)
element_weight = weight_calculator.calculate_weight_at_moisture_content(moisture_content)

print(f"The density of the wood is: {material_density:.2f} kgf/m³")
print(f"The weight of the timber element is: {element_weight:.2f} kgf")

The density of the wood is: 567.16 kgf/m³
The weight of the timber element is: 17.01 kgf


## Section Properties

In this section, we calculate the geometric properties of the rectangular section using the `RectangularSectionPropertie`s class.
An instance of `RectangularSectionProperties` is created with the width and depth of the section.
Then, we calculate the area, moments of inertia, section moduli, and radii of gyration.

In [48]:
# Calculate Section Properties
section_properties = calculation.RectangularSectionProperties(width, depth)
area = section_properties.area()
i_yy = section_properties.moment_of_inertia("yy")
i_zz= section_properties.moment_of_inertia("zz")
elastic_section_modulus_yy = section_properties.elastic_section_modulus("yy")
elastic_section_modulus_zz = section_properties.elastic_section_modulus("zz")
radius_gyration_yy = section_properties.radius_of_gyration("yy")
radius_gyration_zz = section_properties.radius_of_gyration("zz")

print(f"Area of the section: {area:.1f} cm\u00B2")
print(f"Moment of inertia Iyy: {i_yy:.1f} cm\u2074")
print(f"Moment of inertia Izz: {i_zz:.1f} cm\u2074")
print(f"Elastic section modulus Sxx: {elastic_section_modulus_yy:.1f} cm\u00B3")
print(f"Elastic section modulus Syy: {elastic_section_modulus_zz:.1f} cm\u00B3")
print(f"Radius of gyration r_yy: {radius_gyration_yy:.1f} cm")
print(f"Radius of gyration r_zz: {radius_gyration_zz:.1f} cm")

Area of the section: 75.0 cm²
Moment of inertia Iyy: 1406.2 cm⁴
Moment of inertia Izz: 156.2 cm⁴
Elastic section modulus Sxx: 187.5 cm³
Elastic section modulus Syy: 62.5 cm³
Radius of gyration r_yy: 4.3 cm
Radius of gyration r_zz: 1.4 cm


## Effective length

Here, we calculate the effective length of the beam using the `effective_length` function. The effective length is calculated by multiplying the actual length of the beam by the effective length factor (K)

In [50]:
# Calculate Effective Length
effective_length_value = calculation.effective_length(k_factor, length)
print(f"Effective length of the beam: {effective_length_value/100:.2f} m")

Effective length of the beam: 4.00 m


## Combined Adjustment Factors

In this section, we demonstrate how to use the `WoodElementCalculator` class to calculate the combined adjustment factors.
An instance of `WoodElementCalculator` is created with all the adjustment factors defined previously.
Then, the combined factor for each mechanical property (tension, bending, shear, compression, compression perpendicular to the grain, and modulus of elasticity) is calculated.

In [51]:
# Create an instance of WoodElementCalculator
wood_element_calculator = design.WoodElementCalculator(
    tension_factors = ft_factors,
    bending_factors_yy = fb_factors_yy,
    bending_factors_zz = fb_factors_zz,
    shear_factors = fv_factors,
    compression_factors_yy = fc_factors_yy,
    compression_factors_zz = fc_factors_zz,
    compression_perp_factors = fc_perp_factors,
    elastic_modulus_factors = e_factors,
    material_properties = wood_material,
    section_properties = section_properties
)

# Calculate the combined factors
combined_factors = wood_element_calculator.calculate_combined_factors()
print(f"Combined factors: {combined_factors}")

Combined factors: {'tension': np.float64(0.85), 'bending_yy': np.float64(0.85), 'bending_zz': np.float64(0.85), 'shear': np.float64(0.97), 'compression_yy': np.float64(0.8), 'compression_zz': np.float64(0.8), 'compression_perp': np.float64(0.67), 'elastic_modulus': np.float64(0.765)}


## Element Strength

In [52]:
tension_strength = wood_element_calculator.tension_strength()
bending_strength_yy = wood_element_calculator.bending_strength("yy")
bending_strength_zz = wood_element_calculator.bending_strength("zz")
shear_strength = wood_element_calculator.shear_strength()
compression_strength_yy = wood_element_calculator.compression_strength("yy")
compression_strength_zz = wood_element_calculator.compression_strength("zz")
compression_perp_strength = wood_element_calculator.compression_perp_strength()

In [55]:
print(f'The tensile strength is {float(tension_strength)} kgf')
print(f'The bending strength is {float(bending_strength_yy) / 100} kgfm')
print(f'The bending strength is {float(bending_strength_zz) / 100} kgfm')
print(f'The shear strength is {float(shear_strength)} kgf')
print(f'The compression strength yy is {float(compression_strength_yy)} kgf')
print(f'The compression strength zz is {float(compression_strength_zz)} kgf')
print(f'The perpendicular compression strength is {float(compression_perp_strength)} kgf')

The tensile strength is 17850.0 kgf
The bending strength is 446.25 kgfm
The bending strength is 148.75 kgfm
The shear strength is 2182.5 kgf
The compression strength yy is 16800.0 kgf
The compression strength zz is 16800.0 kgf
The perpendicular compression strength is 18.76 kgf


## Ejemplo de funciones para cálculo de todos los elementos

In [22]:
# Factors
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

# Material Properties
species = "Douglas Fir-Larch"
specific_gravity = 0.55
fibre_saturation_point = 28
moisture_content = 19 #19%
e_modulus = 115000 # kgf/cm2
fb = 280  # kgf/cm2
ft = 280  # kgf/cm2a
fv = 45  # kgf/cm2
fc = 280  # kgf/cm2
fc_perp = 28  # kgf/cm2


# Geometry
width = 5. # cm
depth = 15. # cm
length = 400 # cm
k_factor = 1.0

# Geometry
width_2 = 100. # cm
depth_2 = 150. # cm
length_2 = 200 # cm
k_factor_2 = 2.0

# Geometry
width_3 = 15. # cm
depth_3 = 15. # cm
length_3 = 200 # cm
k_factor_3 = 2.0

# Adjustment factors initialization
ft_factors = settings.TensionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_yy = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_zz = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fv_factors = settings.ShearAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.97,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_yy = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_zz = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_perp_factors = settings.PerpendicularAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.67,
    due_temperature=1.0,
    due_bearing_area=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

e_factors = settings.ElasticModulusAdjustmentFactors(
    due_moisture=0.90,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0
)

In [23]:
section = settings.RectangularSection("sección 1", depth=depth, width=width)
section_2 = settings.RectangularSection("sección 2", depth=depth_2, width=width_2)
section_3 = settings.RectangularSection("sección 3", depth=depth_3, width=width_3)
list_sections = [section, section_2, section_3]
list_sections

[RectangularSection(name='sección 1', depth=15.0, width=5.0),
 RectangularSection(name='sección 2', depth=150.0, width=100.0),
 RectangularSection(name='sección 3', depth=15.0, width=15.0)]

In [24]:
element = settings.MemberDefinition("elemento 1", length=length, )
element_2 = settings.MemberDefinition("elemento 2", length=length_2, )
element_3 = settings.MemberDefinition("elemento 3", length=length_3, )
list_members = [element, element_2, element_3]

In [25]:
list_members

[MemberDefinition(name='elemento 1', length=400, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0),
 MemberDefinition(name='elemento 2', length=200, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0),
 MemberDefinition(name='elemento 3', length=200, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0)]

In [26]:
force = settings.Forces(
    name="force_1",
    axial=10000,
    shear_y= 10.0,
    shear_z= 10.0,
    moment_xx = 0.0,
    moment_yy = 0.0,
    moment_zz = 0.0
)

force_2 = settings.Forces(
    name="force_2",
    axial=20000,
    shear_y= 00.0,
    shear_z= 00.0,
    moment_xx = 30.0,
    moment_yy = 400.0,
    moment_zz = 50.0
)
force_3 = settings.Forces(
    name="force_3",
    axial=-10000,
    shear_y= 10.0,
    shear_z= 0.0,
    moment_xx = 0.0,
    moment_yy = 1000.,
    moment_zz = 0.0
)

force_4 = settings.Forces(
    name="force_4",
    axial=-20000,
    shear_y= 10.0,
    shear_z= 20.0,
    moment_xx = 30.0,
    moment_yy = 400.0,
    moment_zz = 50.0
)


In [27]:
list_forces = [force, force_2, force_3, force_4]

In [28]:
design.calculate_dcr_for_wood_elements(
    element= element,
    section= section,
    forces=force,
    material= wood_material,
    tension_factors= ft_factors,
    bending_factors_yy= fb_factors_yy,
    bending_factors_zz= fb_factors_zz,
    shear_factors= fv_factors,
    compression_factors_yy= fc_factors_yy,
    compression_factors_zz= fc_factors_zz,
    compression_perp_factors= fc_perp_factors,
    elastic_modulus_factors=e_factors
)

{'axial tension': 0,
 'tension (dcr)': 0.0,
 'axial compression': 10000,
 'compression (dcr)': 0.5952380952380952,
 'moment yy': 0.0,
 'moment zz': 0.0,
 'biaxial bending (dcr)': 0.0,
 'shear y': 10.0,
 'shear z': 10.0,
 'shear y (dcr)': 0.004581901489117984,
 'shear z (dcr)': 0.004581901489117984,
 'bending and tension (dcr)': 0.0,
 'bending and compression (dcr)': 0.3543083900226757}

# Pruebas


### Prueba para `WeightCalculator.calculate_density_at_moisture_content`

```python



In [29]:
force_results = design.check_for_all_forces(
    section, element, force, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all forces: 100%|██████████| 1/1 [00:00<00:00, 1001.03it/s]

Calculando para la fuerza: force_1


In [30]:
force_results

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 1,force_1,0.595238,0,0.0,10000,0.595238,0.0,0.0,0.0,10.0,10.0,0.004582,0.004582,0.0,0.354308


In [31]:
forces_results = design.check_for_all_forces(
    section, element, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 3495.98it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [32]:
forces_results

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 1,force_1,0.595238,0,0.000000,10000,0.595238,0.0,0.0,0.000000,10.0,10.0,0.004582,0.004582,0.000000,0.354308
1,elemento 1,sección 1,force_2,1.429558,0,0.000000,20000,1.190476,400.0,50.0,0.012325,0.0,0.0,0.000000,0.000000,0.012325,1.429558
2,elemento 1,sección 1,force_3,0.022409,10000,0.560224,0,0.000000,1000.0,0.0,0.022409,10.0,0.0,0.004582,0.000000,0.582633,0.022409
3,elemento 1,sección 1,force_4,0.012325,20000,1.120448,0,0.000000,400.0,50.0,0.012325,10.0,20.0,0.004582,0.009164,1.132773,0.012325


In [33]:
section_result = design.check_for_all_sections(
    section_2, element, force, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/1 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 2
..................................................



Checking for all sections: 100%|██████████| 1/1 [00:00<00:00, 300.00it/s]

Calculando para la fuerza: force_1


In [34]:
section_result

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 2,force_1,0.002976,0,0.0,10000,0.002976,0.0,0.0,0.0,10.0,10.0,0.000023,0.000023,0.0,0.000009


In [35]:
sections_results = design.check_for_all_sections(
    list_sections, list_members, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................



Checking for all forces:   0%|          | 0/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Error processing section 'sección 1': 'list' object has no attribute 'name'
..................................................
Calculando para la sección: sección 2
..................................................



Checking for all forces:   0%|          | 0/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Error processing section 'sección 2': 'list' object has no attribute 'name'
..................................................
Calculando para la sección: sección 3
..................................................



Checking for all sections: 100%|██████████| 3/3 [00:00<00:00, 467.89it/s]

Calculando para la fuerza: force_1
Error processing section 'sección 3': 'list' object has no attribute 'name'

Errors encountered during processing:
Error processing section 'sección 1': 'list' object has no attribute 'name'
Error processing section 'sección 2': 'list' object has no attribute 'name'
Error processing section 'sección 3': 'list' object has no attribute 'name'


In [36]:
sections_results

""


In [37]:
element_results = design.check_for_all_elements(
    list_sections, element, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/1 [00:00<?, ?it/s]

::::::::::::::::::::::::::::::::::::::::::::::::::
Calculando para el elemento: elemento 1
::::::::::::::::::::::::::::::::::::::::::::::::::



cking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 1795.89it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 2
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 3734.91it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 3
..................................................




Checking for all sections: 100%|██████████| 1/1 [00:00<00:00, 56.34it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [38]:
element_results

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 1,force_1,0.595238,0,0.000000,10000,0.595238,0.0,0.0,0.000000,10.0,10.0,0.004582,0.004582,0.000000,0.354308
1,elemento 1,sección 1,force_2,1.429558,0,0.000000,20000,1.190476,400.0,50.0,0.012325,0.0,0.0,0.000000,0.000000,0.012325,1.429558
2,elemento 1,sección 1,force_3,0.022409,10000,0.560224,0,0.000000,1000.0,0.0,0.022409,10.0,0.0,0.004582,0.000000,0.582633,0.022409
3,elemento 1,sección 1,force_4,0.012325,20000,1.120448,0,0.000000,400.0,50.0,0.012325,10.0,20.0,0.004582,0.009164,1.132773,0.012325
4,elemento 1,sección 2,force_1,0.002976,0,0.000000,10000,0.002976,0.0,0.0,0.000000,10.0,10.0,0.000023,0.000023,0.000000,0.000009
5,elemento 1,sección 2,force_2,0.005952,0,0.000000,20000,0.005952,400.0,50.0,0.000005,0.0,0.0,0.000000,0.000000,0.000005,0.000041
6,elemento 1,sección 2,force_3,0.000023,10000,0.002801,0,0.000000,1000.0,0.0,0.000011,10.0,0.0,0.000023,0.000000,0.002812,0.000011
7,elemento 1,sección 2,force_4,0.000046,20000,0.005602,0,0.000000,400.0,50.0,0.000005,10.0,20.0,0.000023,0.000046,0.005608,0.000005
8,elemento 1,sección 3,force_1,0.198413,0,0.000000,10000,0.198413,0.0,0.0,0.000000,10.0,10.0,0.001527,0.001527,0.000000,0.039368
9,elemento 1,sección 3,force_2,0.396825,0,0.000000,20000,0.396825,400.0,50.0,0.003361,0.0,0.0,0.000000,0.000000,0.003361,0.160832


In [39]:
elements_results = design.check_for_all_elements(
    list_sections, list_members, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

::::::::::::::::::::::::::::::::::::::::::::::::::
Calculando para el elemento: elemento 1
::::::::::::::::::::::::::::::::::::::::::::::::::



cking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 1882.96it/s]


...

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 2
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 2006.12it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 3
..................................................




Checking for all sections: 100%|██████████| 3/3 [00:00<00:00, 33.78it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [40]:
elements_results

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 1,force_1,0.595238,0,0.000000,10000,0.595238,0.0,0.0,0.000000,10.0,10.0,0.004582,0.004582,0.000000,0.354308
1,elemento 1,sección 1,force_2,1.429558,0,0.000000,20000,1.190476,400.0,50.0,0.012325,0.0,0.0,0.000000,0.000000,0.012325,1.429558
2,elemento 1,sección 1,force_3,0.022409,10000,0.560224,0,0.000000,1000.0,0.0,0.022409,10.0,0.0,0.004582,0.000000,0.582633,0.022409
3,elemento 1,sección 1,force_4,0.012325,20000,1.120448,0,0.000000,400.0,50.0,0.012325,10.0,20.0,0.004582,0.009164,1.132773,0.012325
4,elemento 1,sección 2,force_1,0.002976,0,0.000000,10000,0.002976,0.0,0.0,0.000000,10.0,10.0,0.000023,0.000023,0.000000,0.000009
5,elemento 1,sección 2,force_2,0.005952,0,0.000000,20000,0.005952,400.0,50.0,0.000005,0.0,0.0,0.000000,0.000000,0.000005,0.000041
6,elemento 1,sección 2,force_3,0.000023,10000,0.002801,0,0.000000,1000.0,0.0,0.000011,10.0,0.0,0.000023,0.000000,0.002812,0.000011
7,elemento 1,sección 2,force_4,0.000046,20000,0.005602,0,0.000000,400.0,50.0,0.000005,10.0,20.0,0.000023,0.000046,0.005608,0.000005
8,elemento 1,sección 3,force_1,0.198413,0,0.000000,10000,0.198413,0.0,0.0,0.000000,10.0,10.0,0.001527,0.001527,0.000000,0.039368
9,elemento 1,sección 3,force_2,0.396825,0,0.000000,20000,0.396825,400.0,50.0,0.003361,0.0,0.0,0.000000,0.000000,0.003361,0.160832


In [41]:
x = design.filter_and_export_results(
    results_df = element_results,
    filters = {
        'section': ['sección 1', 'sección 2'],
        'dcr_max': {'range': {'min': 0.00, 'max': 1.00}},
    },
    sort_by="dcr_max",
    sort_order = "desc"
)

Filtered results exported to: C:\Users\ingah\pycode\timber_nds\tests\test_data\filtered_results.xlsx


In [42]:
x

,member,section,force,dcr_max,axial tension,tension (dcr),axial compression,compression (dcr),moment yy,moment zz,biaxial bending (dcr),shear y,shear z,shear y (dcr),shear z (dcr),bending and tension (dcr),bending and compression (dcr)
0,elemento 1,sección 1,force_1,0.595238,0,0.000000,10000,0.595238,0.0,0.0,0.000000,10.0,10.0,0.004582,0.004582,0.000000,0.354308
2,elemento 1,sección 1,force_3,0.022409,10000,0.560224,0,0.000000,1000.0,0.0,0.022409,10.0,0.0,0.004582,0.000000,0.582633,0.022409
3,elemento 1,sección 1,force_4,0.012325,20000,1.120448,0,0.000000,400.0,50.0,0.012325,10.0,20.0,0.004582,0.009164,1.132773,0.012325
5,elemento 1,sección 2,force_2,0.005952,0,0.000000,20000,0.005952,400.0,50.0,0.000005,0.0,0.0,0.000000,0.000000,0.000005,0.000041
4,elemento 1,sección 2,force_1,0.002976,0,0.000000,10000,0.002976,0.0,0.0,0.000000,10.0,10.0,0.000023,0.000023,0.000000,0.000009
7,elemento 1,sección 2,force_4,0.000046,20000,0.005602,0,0.000000,400.0,50.0,0.000005,10.0,20.0,0.000023,0.000046,0.005608,0.000005
6,elemento 1,sección 2,force_3,0.000023,10000,0.002801,0,0.000000,1000.0,0.0,0.000011,10.0,0.0,0.000023,0.000000,0.002812,0.000011
